# User defined inputs - please update as instructed

In [1]:
"""In this code block, you only need to update the values for the variables galaxy, 
  halpha_inputfilename, cont2_inputfilename, cont1_inputfilename, 
  and input_muse_filename to match your specific requirements."""

# User Input: Define the galaxy
galaxy = 'ngc628'

# User Input: Define input files for continuum subtraction
halpha_inputfilename = '../hst_cropped/ngc628_acs_wfc_f658n_sci_regrid_shifted.fits'
cont2_inputfilename = '../hst_cropped/ngc628_acs_f814w_exp_drc_sci.fits'
cont1_inputfilename = '../hst_cropped/ngc628_uvis_f555w_exp_drc_sci.fits'

# User Input: Define input files for flux postprocessing
input_muse_filename = '/Users/abarnes/Dropbox/work/Projects/pressures/phangs/data/maps/muse/NGC0628-0.92asec_MAPS.fits'

# The following should run automatically with little/no user input

In [2]:
import os
import astropy.units as u
from astropy.io import fits
from reduction_phangs_hst import contsub, contsub_misc, contsub_postprocess
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Create output directory for continuum subtraction
# The output directory will be named based on the substring extracted from the first and second continuum filenames
cont1_substring = contsub_misc.extract_substring(cont1_inputfilename)  # Extract substring from the first continuum filename
cont2_substring = contsub_misc.extract_substring(cont2_inputfilename)  # Extract substring from the second continuum filename
output_dir = './%s_%s/' % (cont1_substring, cont2_substring)  # Define the output directory path
contsub_misc.create_directory(output_dir)  # Create the output directory if it doesn't exist

halpha_filename = '%s/%s_halpha_raw.fits' % (output_dir, galaxy)  # Set the output filename for the continuum-subtracted emission line image
cont_filename = '%s/%s_cont_raw.fits'  % (output_dir, galaxy)  # Set the output filename for the scaled continuum image

# Run continuum subtraction
# Perform continuum subtraction on the halpha_inputfilename using the cont1_inputfilename and cont2_inputfilename
contsub.continuum_subtraction(halpha_inputfilename, cont1_inputfilename, halpha_filename, cont_filename, cont2_inputfilename)

Directory './f555w_f814w/' already exists.
[INFO] Loading Continuum 1: ../hst_cropped/ngc628_uvis_f555w_exp_drc_sci.fits
[INFO] Loading Emission Line Image: ../hst_cropped/ngc628_acs_wfc_f658n_sci_regrid_shifted.fits
[INFO] Loading Continuum 2: ../hst_cropped/ngc628_acs_f814w_exp_drc_sci.fits
[INFO] Continuum file saved: ./f555w_f814w//ngc628_cont_raw.fits.fits


In [4]:
# Define the output filename by replacing '_raw.fits' with '.fits'
output_filename = halpha_filename.replace('_raw.fits', '.fits')

# Process the Halpha units and save the result to the output file
hdu_hst = contsub_postprocess.process_halpha_units(halpha_filename, output_filename)

Processing ./f555w_f814w//ngc628_halpha_raw.fits
Scaling the data...
Converting units...
Saving the processed FITS file as ./f555w_f814w//ngc628_halpha.fits


In [5]:
# Define the output MUSE filename using the output directory and galaxy name
output_muse_filename = '%s/%s_musehalpha.fits' % (output_dir, galaxy)

# Process the Halpha MUSE file and save the result to the output file
hdu_muse = contsub_postprocess.process_halpha_muse(input_muse_filename, output_muse_filename)

Processing Halpha MUSE file: /Users/abarnes/Dropbox/work/Projects/pressures/phangs/data/maps/muse/NGC0628-0.92asec_MAPS.fits


In [6]:
# Define the output MUSE regrid filename using the output directory and galaxy name
output_muse_filename = '%s/%s_musehalpha_regrid.fits' % (output_dir, galaxy)

# Perform regridding of the MUSE data using the HST data and save the result to the output file
hdu_muse_regrid = contsub_postprocess.regrid(hdu_muse, hdu_hst, output_muse_filename)

[INFO] Reprojecting the input image to match the template WCS...
[INFO] Performing image reprojection...
[INFO] Image reprojection complete.
[INFO] Scaling the output data to conserve flux with factor 0.04
[INFO] Flux scaling complete.
[INFO] Saving the reprojected image to: ./f555w_f814w//ngc628_musehalpha_regrid.fits
[INFO] Image saved successfully.
[INFO] Reprojection process completed.


In [7]:
# Set the initial resolution for HST observations
initial_resolution = 0.1 * u.arcsec

# Extract the resolution substring from the input_muse_filename for the desired resolution
desired_resolution = float(contsub_misc.extract_substring(input_muse_filename, pattern=r'\d+\.\d+')) * u.arcsec

# Smooth the HST image with the desired beam resolution and save the result to the output file
output_filename = halpha_filename.replace('_raw.fits', '_smoothed.fits')
hdu_hst_smoothed = contsub_postprocess.smooth_image_with_beam(hdu_hst, initial_resolution, desired_resolution, output_filename=output_filename)

[INFO] Smoothing the image with a beam kernel...
[INFO] Pixel scale: 0.04 arcsec arcsec
[INFO] Initial Resolution: 0.10 arcsec arcsec
[INFO] Desired Resolution: 0.92 arcsec arcsec
[INFO] Convolution kernel created.
[INFO] Performing image convolution...
[INFO] Image convolution complete.
[INFO] Saving the smoothed image to: ./f555w_f814w//ngc628_halpha_smoothed.fits
[INFO] Image saved successfully.
[INFO] Smoothing process completed.


In [8]:
# Generate the output filename for the ratio image by replacing '_raw.fits' with '_hstmuseratio.fits'
output_ratio_filename = halpha_filename.replace('_raw.fits', '_hstmuseratio.fits')

# Generate the output filename for the anchored HST image by replacing '_raw.fits' with '_anchored.fits'
output_anchored_filename = halpha_filename.replace('_raw.fits', '_anchored.fits')

# Save the ratio image and smoothed HST image, and obtain the resulting HDUs
hdu_ratio_smooth, hdu_hst_anchored = contsub_postprocess.save_ratio_smoothed_image(hdu_muse_regrid, hdu_hst, hdu_hst_smoothed, output_ratio_filename, output_anchored_filename)

[INFO] Ratio smoothed image saved as ./f555w_f814w//ngc628_halpha_hstmuseratio.fits
[INFO] Anchored HST image saved as ./f555w_f814w//ngc628_halpha_anchored.fits


In [9]:
# Generate the output filename for the anchored HST image with intensity negations
output_anchored_filename = halpha_filename.replace('_raw.fits', '_anchored_intnegs.fits')

# Process the anchored HST image with intensity negations and save the result to the output file
hdu_hst_anchored_intnegs = contsub_postprocess.process_anchored_image(hdu_hst_anchored, output_anchored_filename)

[INFO] Anchored HST image with negative values processed and saved as ./f555w_f814w//ngc628_halpha_anchored_intnegs.fits


In [10]:
# Generate the output filename for the anchored HST image with added white noise
output_filename = halpha_filename.replace('_raw.fits', '_anchored_wnoise.fits')

# Add white noise to the anchored HST image and save the result to the output file
hdu_ha_hst_anchored_wnoise = contsub_postprocess.add_noise_to_image(hdu_hst_anchored, output_filename)

# Generate the output filename for the anchored HST image with intensity negations and added white noise
output_filename = halpha_filename.replace('_raw.fits', '_anchored_intnegs_wnoise.fits')

# Add white noise to the anchored HST image with intensity negations and save the result to the output file
hdu_hst_anchored_intnegs_wnoise = contsub_postprocess.add_noise_to_image(hdu_hst_anchored, output_filename)

[INFO] Image with added noise saved as ./f555w_f814w//ngc628_halpha_anchored_wnoise.fits
[INFO] Image with added noise saved as ./f555w_f814w//ngc628_halpha_anchored_intnegs_wnoise.fits
